In [36]:
import json
import random
import numpy as np
from tqdm import tqdm
import pandas as pd

In [24]:
root = '.'
train_data = list(
    map(json.loads, open(f"{root}/cosmos/train_data.json").readlines())
)
test_data = list(
    map(json.loads, open(f"{root}/cosmos/test_data.json").readlines())
)

In [53]:
test_data = list(
    map(json.loads, open(f"{root}/cosmos/test_data_2.json").readlines())
)

JSONDecodeError: Extra data: line 1 column 815 (char 814)

In [31]:
train_data[0].keys()

dict_keys(['img_local_path', 'articles', 'maskrcnn_bboxes'])

In [54]:
# Take 2 random correct caption
def gen_positive_case(data):
    caption_1_idx, caption_2_idx = random.sample(range(0,len(data['articles'])),2)
    cap1 = data['articles'][caption_1_idx]['caption_modified']
    cap2 = data['articles'][caption_2_idx]['caption_modified']
    if cap1 == cap2:
        return None
    return [data['img_local_path'],cap1,cap2,False,data['maskrcnn_bboxes']]

In [55]:
# Take 2 random false caption
def gen_negative_case_1(data, train_data):
    data_random1 = train_data[np.random.randint(len(train_data))]
    while data['img_local_path'] == data_random1['img_local_path']:
        data_random1 = train_data[np.random.randint(len(train_data))]
    cap1 = data_random1['articles'][np.random.randint(len(data_random1['articles']))]['caption_modified']
    data_random2 = train_data[np.random.randint(len(train_data))]
    while data['img_local_path'] == data_random2['img_local_path']:
        data_random2 = train_data[np.random.randint(len(train_data))]
    cap2 = data_random2['articles'][np.random.randint(len(data_random2['articles']))]['caption_modified']
    return [data['img_local_path'],cap1,cap2,True,data['maskrcnn_bboxes']]

In [56]:
np.random.seed(42)
# train_data_sample = np.random.choice(train_data, size=int(len(train_data)*50/100))
# Not OOC cases
l = []

for data in tqdm(train_data):
    if len(data['articles'])>1:
        g=gen_positive_case(data)
        if g is not None:
            l.append(g)
    z = gen_negative_case_1(data, train_data)
    if z is not None:
        l.append(z)

100%|██████████| 161754/161754 [00:05<00:00, 31548.18it/s]


In [57]:
dataframe = pd.DataFrame(
    l, columns=["img_local_path", "caption1","caption2","context_label","maskrcnn_bboxes"],
)

In [66]:
s = dataframe[:20].to_json(orient='records').replace('},','}\n')[1:-1]

In [67]:
with open('cosmos/test_data_2.json', "w") as text_file:
    text_file.write(s)